In [ ]:
import pandas as pd

In [ ]:
# directory = '/media/share/DataSaves/wikitable_extract_220814_test'
directory = '/Volumes/14TB/DataSaves/wikitable_extract_220814_test'

In [ ]:
import glob
test = glob.glob(directory + '/*.parquet')

In [ ]:
len(test)

In [ ]:
# ! pip inst?all py-wikimarkup

In [ ]:
from wikimarkup.parser import Parser
from bs4 import BeautifulSoup

parser = Parser()

def get_tables(markdowntext):
    output = []
    try:
        html = parser.parse(markdowntext)
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('table')
        df_tables = pd.read_html(html)
        if len(tables) != len(df_tables):
            print("tables offset..")
            return []
        output = []
        for i, table in enumerate(tables):
            # all previous headings in order
            headings = []
            for k in range(6):
                h = table.find_previous_sibling(f'h{k}')
                headings.append(
                    h.text if h else ''
                )
            output.append((i, "\n".join(headings), df_tables[i]))
    except Exception as e:
        print("Failed to parse...")
    return output

def get_all_tables(path):
    data = pd.read_parquet(path)
    data = data['page_text'].groupby(data['page_title_text']).first()
    for title, text in data.iteritems():
        result = get_tables(text)
        for row in result:
            yield title, *row

In [ ]:
from sketch import Portfolio

In [ ]:
import time
st = time.time()
results = []
for k, thing in enumerate(test[4:54]):
    print(k, thing)
    for i, x in enumerate(get_all_tables(thing)):
        results.append(x)
        if i%100 == 0:
            print(i, time.time() - st)
            pf = Portfolio()
            for result in results:
                pf.add_wikitable(result[0], result[1], result[2], result[3])
            print(f"About to upload... {len(pf.sketchpads)}")
            pf.upload(apiKey="8f79be2b6d0d47ccb8192e46f38c80ce")
            results = []
            

In [ ]:
i

In [ ]:
# there's an infinite loop somewhere in here, things keep hanging